In [4]:
import pandas as pd
import re
from konlpy.tag import Okt

In [5]:
combined_df = pd.read_csv("C:/Users/82105/Downloads/combined.csv", encoding="utf-8-sig")

In [3]:
# 전처리를 적용할 컬럼을 확인합니다. 'input'과 'output' 컬럼이 중요해 보입니다.
# 혹시 데이터가 없는 경우(NaN)를 대비해 빈 문자열로 채워줍니다.
combined_df['input'] = combined_df['input'].fillna('')
combined_df['output'] = combined_df['output'].fillna('')

In [4]:
# 모델 3 : 답변 검색 모델을 위한 '통합' 텍스트 컬럼 생성
combined_df['text_combined'] = combined_df['input'].astype(str) + " " + combined_df['output'].astype(str)

# 1단계 : KoNLPy를 사용한 정교한 전처리

In [5]:
# 1. 사용자 정의 불용어 리스트: 이 리스트에 있는 단어는 무조건 제거됩니다.
#  추가로 의미 없게 자주 등장하는 단어가 있다면 리스트에 계속 추가해 나가시면 더욱 정교한 분석이 가능할 것
# 분석하며 계속 추가해나갈 수 있습니다.
KOREAN_STOPWORDS = [
    # 일반적인 한글 불용어
    '이', '그', '저', '것', '수', '등', '들', '더', '없', '있', '하', '되', '않', 
    '나', '우리', '너', '당신', '같', '또', '만', '수', '것', '때', '등', '년', '월', '일',
    '때문', '정도', '사실', '생각', '경우', '문제', '방법', '상황', '내용', '결과',

    # 법률 도메인 일반 불용어  
    '가능하다', '가능', '가다', '되다', '하다', '있다', '없다', '않다', '된다', '한다',
    '어떻게', '어떤', '무엇', '언제', '어디서', '왜', '누가', '얼마', '몇',
    '알고', '싶다', '궁금하다', '문의', '질문', '답변', '설명', '이해',
    '과정', '절차', '이후', '다음', '먼저', '그리고', '그러나', '하지만', '그래서', '제자',

    # 법률 질문 패턴 불용어
    '해야', '하면', '경우', '때는', '어느', '무슨', '어디', '누구' , ' 가지다' , '가지'
]

In [6]:
def protect_term(match):
    # 찾은 문자열(예: '제3자')을 고유한 토큰(예: '###TOKEN0###')으로 매핑
    token = f"###TOKEN{len(protected_matches)}###"
    protected_matches[token] = match.group(0)
    return token

def preprocess_text(text):  # 전처리 함수 
    """
    한국어 텍스트를 입력받아 전처리하는 함수:
    1. '제3자' 형태의 단어를 임시 토큰으로 보호/복원하여 숫자를 보존.
    2. 불필요한 한글 이외의 문자를 제거.
    3. 형태소 분석 및 어간 추출.
    4. 불용어 및 1글자 단어 제거.
    """
    # 1단계: 한글, 공백을 제외한 모든 문자 제거
    # re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', text)는 한글과 공백만 남기고 나머지는 지우라는 의미
    # 제숫자 + 한글 글자 + 선택적 조사까지 포함
    # 딕셔너리를 함수 내부에 선언하여 매 호출마다 초기화 (핵심 수정 사항)
    protected_matches = {}

    def protect_term(match):
        # 찾은 문자열(예: '제3자')을 고유한 토큰(예: '###TOKEN0###')으로 매핑
        token = f"###TOKEN{len(protected_matches)}###"
        protected_matches[token] = match.group(0)
        return match.group(0) # 일단 원본 단어 그대로 둔 채로 형태소 분석 진행


    # 1단계: '제3자' 형태의 단어를 임시 토큰으로 치환하여 보호
    re.sub(r'(제\d+[가-힣]+)', protect_term, text)

    # 2단계: 한글, 공백, 보호 토큰의 문자(#)만 남기고 제거
    text = re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣# ]', '', text)

    # 2단계: Okt 형태소 분석기를 이용한 토큰화 및 어간 추출(Stemming)
    # okt.pos(text, stem=True)는 문장을 (단어, 품사) 형태로 나누고, '하다', '먹다'처럼 원형으로 만들어줍니다.
    # 예: "먹었었고" -> ('먹다', 'Verb')
    # Okt 형태소 분석기 객체 생성
    okt = Okt()
    word_tokens = okt.pos(text, stem=True)

    # 3단계: 불용어 제거
    # 형태소 분석 후 의미 있는 단어만 추출하는 필터링 과정
    # 의미를 가지는 명사, 동사, 형용사, 부사 중에서 1글자 이상인 단어만 추출합니다.
    # 품사 태그가 'Josa'(조사), 'Eomi'(어미), 'Punctuation'(구두점) 등인 단어들을 제거합니다.
    meaningful_words = []
    for word, pos in word_tokens:
        # 1글자 단어 필터링은 의미있는 품사에서만 수행
        is_meaningful_pos = pos in ['Noun', 'Verb', 'Adjective', 'Adverb']
        
        if is_meaningful_pos:
            # 형태소 분석된 결과 그대로 '제3자'는 Noun으로 분류될 가능성이 높으며, 길이가 3입니다.
            if len(word) > 1 and word not in KOREAN_STOPWORDS:
                meaningful_words.append(word)
        # '제3자'와 같이 숫자+한글이 붙은 단어는 1글자가 아니므로 포함됩니다.
   
            
     # 6. 보호된 단어 강제 포함 및 복원 (핵심)
    # 보호 목록에 있던 단어들을 강제로 최종 리스트에 추가합니다.
    # (이미 리스트에 분해되어 들어갔을 수 있지만, 완벽한 복원을 위해 강제 추가)
    for original_term in protected_matches.values():
        # '제3자' 자체를 하나의 단어로 명시적으로 추가
        meaningful_words.append(original_term)
        
    # 7. 중복 제거 및 최종 문자열 반환
    # Set을 이용해 중복 제거 후 리스트로 변환
    final_words = list(set(meaningful_words))
    # 최종적으로 공백으로 연결된 문자열을 반환합니다. 
    # 모델에 따라 리스트 형태(meaningful_words)를 그대로 사용할 수도 있습니다.
    # 텍스트에서 분석에 의미 있는 핵심 단어들만 남긴 리스트 생성
    return ' '.join(final_words)

In [7]:
# [목적 2] 답변 검색 모델용: 'text_combined' 컬럼을 전처리
combined_df['combined_processed'] = combined_df['text_combined'].apply(preprocess_text)

In [6]:
# 결과 비교를 위해 원본과 처리된 결과를 나란히 출력
print("\n=== 전처리 전/후 비교 ===")
for i in range(5):
    print(f"원본 [{i}]: {combined_df['text_combined'].iloc[i]}")
    print(f"결과 [{i}]: {combined_df['combined_processed'].iloc[i]}\n")


# 전처리된 데이터가 포함된 데이터프레임 확인
print("=== 각 목적에 맞게 생성된 전처리 컬럼들 ===")
print(combined_df[['combined_processed']].head())


=== 전처리 전/후 비교 ===
원본 [0]: 혼인 중 발생한 금전거래와 관련하여 차용금 반환 청구권이 인정되는 조건은 무엇인가요? 혼인 중 발생한 금전거래에 대한 차용금 반환 청구권이 인정되기 위해서는 차용금 지급의 사실을 입증할 객관적인 금융거래 자료가 필요합니다. 또한 차용금의 변제를 독촉하는 증거가 존재해야 하며, 재산분할 관련 약정에서 명확하게 차용금에 대한 정함이 있어야 합니다.
결과 [0]: 명확하다 인정 반환 조건 지급 금융 정함 또한 자료 필요하다 독촉 금전 증거 변제 존재 재산 관련 분할 혼인 약정 차용 대한 객관 발생 입증 청구권 거래

원본 [1]: 제3자가 부부의 일방과 부정행위를 할 경우 어떤 법적 책임을 지게 되나요? 제3자는 타인의 부부공동생활에 개입하여 그 파탄을 초래하거나 본질적인 부부공동생활을 방해하여서는 안 됩니다. 제3자가 부부의 일방과 부정행위를 함으로써 부부공동생활을 침해하고 배우자에게 정신적 고통을 주는 행위는 원칙적으로 불법행위에 해당합니다. 따라서 제3자는 불법행위책임으로 인해 위자료를 지급할 의무가 있습니다.
결과 [1]: 방해 정신 초래 인하다 타인 불법행위 따라서 지급 해당 지게 부정행위 본질 책임 침해 행위 부부 원칙 생활 파탄 의무 제3자가 주다 제3자는 배우자 고통 위자료 일방 거나 서다 법적 어떻다 개입

원본 [2]: 민법 제840조 제1호에서 규정한 재판상 이혼사유 중 부정한 행위의 의미는 무엇인가요? 민법 제840조 제1호에 따른 재판상 이혼사유인 부정한 행위는 간통에 이르지 아니하였더라도 부부의 정조의무에 충실하지 않은 일체의 부정행위를 포함하는 넓은 개념으로 해석됩니다. 대법원 판례에 따르면 일방 배우자가 배우자 있는 다른 사람과 부정행위를 저지른 경우, 이는 이혼 사유로 인정될 수 있습니다.
결과 [2]: 제1호에서 해석 인정 일체 제840조 제조 이다 민법 넓다 저지르다 의미 충실하다 아니다 이르다 부정행위 이혼 따르다 행위 부부 의무 대법원 간통 사람과 개념 배우자 일방 제1호에 포함

In [7]:
combined_df['topic_label'] = -1  # 초기값 -1로 설정

# 데이터 확인
print(f"데이터 크기: {len(combined_df)} 행")
print(f"컬럼: {list(combined_df.columns)}")
print(f"topic_label 초기값: {combined_df['topic_label'].value_counts()}")

데이터 크기: 4447 행
컬럼: ['doc_id', 'casetype', 'casenames', 'input', 'output', 'fileName', 'announce_date', 'decision_date', 'response_date', 'is_divorce', 'input_processed', 'topic_label', 'text_combined', 'combined_processed']
topic_label 초기값: topic_label
-1    4447
Name: count, dtype: int64


In [2]:
combined_df.to_csv("C:/Users/82105/Downloads/combined.csv", index=False, encoding='utf-8-sig')

NameError: name 'combined_df' is not defined

# 2단계: 최적화된 TF-IDF 벡터화

In [9]:
# 텍스트 데이터를 머신러닝 알고리즘이 처리할 수 있는 수치 벡터로 변환
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


# TF-IDF Vectorizer 설정
vectorizer = TfidfVectorizer(
    max_features=5000,      # 상위 빈도 1000개 단어만 사용
    min_df=2,              # 최소 2개 문서에 나타나야 포함
    max_df=0.7,           # 80% 이상 문서에 나타나는 단어 제거
    lowercase=False,         # 소문자 변환 x
    ngram_range=(1, 2),     # 1-gram, 2-gram 모두 사용
    sublinear_tf=True,            # TF 값에 로그 스케일 적용 (성능 향상)
    token_pattern=r'[가-힣]{2,}', # 한글 2글자 이상
)

# TF-IDF 행렬 생성
tfidf_matrix = vectorizer.fit_transform(combined_df['combined_processed'])  # X는 (문서 수 × 단어 수) 크기의 희소 행렬

print(f"TF-IDF 행렬 크기: {tfidf_matrix.shape}")
print(f"문서 수: {tfidf_matrix.shape[0]}")
print(f"특성(단어) 수: {tfidf_matrix.shape[1]}")

# 특성 이름(단어들) 확인
# vectorizer.get_feature_names_out()로 어떤 단어가 벡터에 들어갔는지 확인 가능
feature_names = vectorizer.get_feature_names_out()
print(f"상위 10개 특성: {feature_names[:10]}")
print(f"상위 20개 특성: {feature_names[:20]}")

TF-IDF 행렬 크기: (4447, 5000)
문서 수: 4447
특성(단어) 수: 5000
상위 10개 특성: ['가능성' '가능성 결정' '가능성 위자료' '가능성 피해' '가동' '가사' '가사소송법' '가압류' '가압류 간주'
 '가압류 기각']
상위 20개 특성: ['가능성' '가능성 결정' '가능성 위자료' '가능성 피해' '가동' '가사' '가사소송법' '가압류' '가압류 간주'
 '가압류 기각' '가압류 명시' '가압류 불법행위' '가압류 산정' '가압류 선고' '가압류 의거' '가압류 제조' '가압류 조건'
 '가압류 중단' '가압류 채권' '가압류 통상']


# 3단계: 토픽 개수 최적화
# 최적의 토픽 개수를 찾기 위해 여러 지표를 사용

In [13]:
# 4-1. LDA Perplexity 계산 (낮을수록 좋음)

# 토픽 개수별 Perplexity 계산
perplexity_scores = []
topic_range = range(2, 8)  # 2~10개 토픽 테스트

for n_topics in topic_range:
    lda = LatentDirichletAllocation(
        n_components=n_topics, 
        random_state=42,
        max_iter=100,
        doc_topic_prior=0.1,      # 문서-토픽 분포 조정
        topic_word_prior=0.01     # 토픽-단어 분포 조정
    )
    lda.fit(tfidf_matrix)
    perplexity = lda.perplexity(tfidf_matrix)
    perplexity_scores.append(perplexity)
    print(f"토픽 수 {n_topics}: Perplexity = {perplexity:.4f}")

# 최적의 토픽 수 선택
optimal_topics = topic_range[np.argmin(perplexity_scores)]
print(f"최적의 토픽 수 (Perplexity 기준): {optimal_topics}")

토픽 수 2: Perplexity = 6325.4513
토픽 수 3: Perplexity = 6982.2974
토픽 수 4: Perplexity = 7436.3214
토픽 수 5: Perplexity = 8321.9306
토픽 수 6: Perplexity = 8012.8821
토픽 수 7: Perplexity = 9517.6950
최적의 토픽 수 (Perplexity 기준): 2


In [14]:
# 4-2. K-Means Elbow Method (WCSS)

# K-Means의 WCSS 계산
wcss_scores = []
k_range = range(1, 8)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(tfidf_matrix)
    wcss_scores.append(kmeans.inertia_)
    print(f"K={k}: WCSS={kmeans.inertia_:.4f}")

# Elbow Point 시각화로 확인 가능

K=1: WCSS=4292.7656
K=2: WCSS=4237.5112
K=3: WCSS=4189.1095
K=4: WCSS=4168.0205
K=5: WCSS=4149.2559
K=6: WCSS=4119.1668
K=7: WCSS=4105.0661


# 더 정확한 토픽 모델링을 위해 Gensim 라이브러리를 사용
# 고급 기법: Gensim LDA + Coherence Score

In [10]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models import CoherenceModel

# 텍스트를 토큰 리스트로 변환
texts = [text.split() for text in combined_df['combined_processed']]

# Dictionary 생성
dictionary = Dictionary(texts)
dictionary.filter_extremes(no_below=2, no_above=0.8)

# Corpus 생성  
corpus = [dictionary.doc2bow(text) for text in texts]

# 최적 토픽 수 찾기 (Coherence Score 사용)
coherence_scores = []
topic_range = range(2, 11)

for num_topics in topic_range:
    lda_model = LdaModel(
        corpus=corpus,
        id2word=dictionary, 
        num_topics=num_topics,
        random_state=42,
        passes=10
    )
    
    coherence_model = CoherenceModel(
        model=lda_model, 
        texts=texts, 
        dictionary=dictionary, 
        coherence='c_v'
    )
    
    coherence_score = coherence_model.get_coherence()
    coherence_scores.append(coherence_score)
    print(f"토픽 수 {num_topics}: Coherence = {coherence_score:.4f}")

optimal_topics = topic_range[np.argmax(coherence_scores)]
print(f"최적 토픽 수 (Coherence 기준): {optimal_topics}")

토픽 수 2: Coherence = 0.5749
토픽 수 3: Coherence = 0.5218
토픽 수 4: Coherence = 0.5380
토픽 수 5: Coherence = 0.4985
토픽 수 6: Coherence = 0.5457
토픽 수 7: Coherence = 0.5722
토픽 수 8: Coherence = 0.5894
토픽 수 9: Coherence = 0.5492
토픽 수 10: Coherence = 0.5496
최적 토픽 수 (Coherence 기준): 8
